In [1]:
from arcgis.gis import GIS
from arcgis import geocoding

import sqlite3
from osgeo import ogr
import pandas as pd

from passwords import arcgis

user = arcgis['user']
password = arcgis['password']

db = 'C:\\Users\\Sean Tobin\\Documents\\Data backups\\___LaraScrape___.sqlite'

#### While working with the whole MSA, the number of coordinates to geocode is a little large for the ?computer? to handle

So we're writing each coordinate to text file with the ID and the Coordinate. I'll forgo appending to output list to create pd dataframe in one go. 

In [2]:
coords_list = pd.read_csv('coords_list.txt', header = None)
print(len(coords_list))
success_ids = coords_list[0].to_list()
skip_these = "('" + str(success_ids[0]) + "','" + "','".join([str(x) for x in success_ids[1:]])+"')"

183239


In [3]:
columns = ['ID_Number','EntityName','EntityType','OrganisationDate','InactiveDate','RevivalDateText','MostRecentAnnualReportYear', 'Resident_addr']

conn = sqlite3.connect(db)
c = conn.cursor()
c.execute('''Select
                    DISTINCT ID_Number,
                    EntityName,
                    EntityType,
                    OrganisationDate,
                    InactiveDate,
                    RevivalDateText,
                    MostRecentAnnualReportYear,
                    ResidentStreet ||" " || ResidentCity ||" "|| ResidentState||" " || ResidentZip as Resident_addr
                FROM LaraBusinessClean
                Where ResidentCity in ('AUBURN HILLS','BERKLEY','BIRMINGHAM','BLOOMFIELD HILLS','CLARKSTON','CLAWSON','FARMINGTON',
                             'FARMINGTON HILLS','FENTON','FERNDALE','HAZEL PARK','HUNTINGTON WOODS','KEEGO HARBOR','LAKE ANGELUS',
                             'LATHRUP VILLAGE','MADISON HEIGHTS','NORTHVILLE','NOVI','OAK PARK','ORCHARD LAKE VILLAGE','PLEASANT RIDGE',
                             'PONTIAC','ROCHESTER','ROCHESTER HILLS','ROYAL OAK','SOUTH LYON','SOUTHFIELD','SYLVAN LAKE',
                             'TROY','WALLED LAKE','WIXOM','CENTER LINE','EASTPOINTE','EAST DETROIT','FRASER','GROSSE POINTE SHORES',
                             'MEMPHIS','MOUNT CLEMENS','NEW BALTIMORE','RICHMOND','ROSEVILLE','ST. CLAIR SHORES','STERLING HEIGHTS',
                             'UTICA','WARREN','BRIGHTON','FENTON','HOWELL','ALGONAC','MARINE CITY','MARYSVILLE','MEMPHIS','PORT HURON',
                             'RICHMOND','ST. CLAIR','YALE','BROWN CITY','IMLAY CITY','LAPEER','ALLEN PARK','BELLEVILLE','DEARBORN HEIGHTS',
                             'DEARBORN','DETROIT','ECORSE','FLAT ROCK','GARDEN CITY','GIBRALTAR','GROSSE POINTE','GROSSE POINTE FARMS',
                             'GROSSE POINTE PARK','GROSSE POINTE SHORES','GROSSE POINTE WOODS','HAMTRAMCK','HARPER WOODS','HIGHLAND PARK',
                             'INKSTER','LINCOLN PARK','LIVONIA','MELVINDALE','NORTHVILLE','PLYMOUTH','RIVER ROUGE','RIVERVIEW','ROCKWOOD',
                             'ROMULUS','SOUTHGATE','TAYLOR','TRENTON','WAYNE','WESTLAND','WOODHAVEN','WYANDOTTE')
                        AND ID_Number NOT IN ''' + skip_these)  

df = c.fetchall()
conn.close()

data = pd.DataFrame(df, columns = columns)
print(len(data))
data.head()

555443


,ID_Number,EntityName,EntityType,OrganisationDate,InactiveDate,RevivalDateText,MostRecentAnnualReportYear,Resident_addr
0,800393976,"ROSEDALE PARK INVESTORS, INC.",DOMESTIC PROFIT CORPORATION,04/30/1991,09/05/2019,\n,2019,19395 STRATHCONA DETROIT MI 48203
1,800393979,"TELECOM TECH, INC.",DOMESTIC PROFIT CORPORATION,03/12/1996,07/15/1999,\n,,522 SMITH BIRMINGHAM MI 48009
2,800393980,"IWC INVESTMENTS, INC.",DOMESTIC PROFIT CORPORATION,03/24/2000,07/15/2010,\n,2007,12838 GAVEL DETROIT MI 48227
3,800393982,"MORGAN'S GARAGE, INC.",DOMESTIC PROFIT CORPORATION,03/12/1996,None,\n,2019,828 SYMES ROYAL OAK MI 48067
4,800393986,"WESTWOOD IMPORTING COMPANY OF MICHIGAN, INC.",DOMESTIC PROFIT CORPORATION,03/12/1996,07/15/1999,\n,,28411 NORTHWESTERN HWY #925 SOUTHFIELD MI 48034


In [4]:
my_gis = GIS(url = 'http://detroitmi.maps.arcgis.com/', username= user, password = password)
# List GeoCoders available to me 
GeoCode = geocoding.get_geocoders(my_gis)
geocoding.get_geocoders(my_gis)

[<Geocoder url:"http://gis.detroitmi.gov/arcgis/rest/services/DoIT/CompositeGeocoder/GeocodeServer">,
 <Geocoder url:"http://gis.detroitmi.gov/arcgis/rest/services/DoIT/AddressPointGeocoder/GeocodeServer">,
 <Geocoder url:"http://gis.detroitmi.gov/arcgis/rest/services/DoIT/StreetCenterlineGeocoder/GeocodeServer">,
 <Geocoder url:"https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer">]

In [5]:
bar = data.Resident_addr.to_list()

In [6]:
len(data.Resident_addr)

555443

In [ ]:
addresses = data['Resident_addr']
ID_numbers = data['ID_Number']

#Output_list = [] Think this might be why the process eventually fails.

# Create a point
point = ogr.Geometry(ogr.wkbPoint)

# Open file to dump the coords into with the business ID.
with open('coords_list.txt',"a") as filehandle:
    
    for address, ID in zip(addresses,ID_numbers):
        test_coords = geocoding.geocode(address, geocoder = GeoCode[-1])

        # Grab a list of scores from the test_coords
        score = [x['score'] for x in test_coords]

        # Use index of max score to create OGR Point object
        point.AddPoint(test_coords[score.index(max(score))]['location']['x'],test_coords[score.index(max(score))]['location']['y'])
        
        # Append new wkt point to output
        #Output_list.append(point.ExportToWkt())
        filehandle.write('%s\n' % str(ID + ',' + point.ExportToWkt()))